# Combine Movie Functions

In [8]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from wikipedia import wikipedia
from wikifilm import wikifilm
from oscar import oscars
from razzies import razzies
from boxofficemojo import boxofficemojo
from rottentomatoes import rottentomatoes
from ultimatemovierankings import ultimatemovierankings
from combine import combine


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-01-06 00:13:28.703593


# Get/Parse/Merge/Process Rotten Tomatoes Data

In [15]:
%load_ext autoreload
%autoreload

comb = combine()
comb.setOscarData(oscars(wikipedia()), 100)
comb.setWikiFilmData(wikifilm(), 1)
comb.setRazziesData(razzies(), 1)
comb.setRottenTomatoesData(rottentomatoes(), 95)
comb.setBoxOfficeMojoData(boxofficemojo(), 100e6)
comb.setUltimateMovieRankingsData(ultimatemovierankings(), 90)
comb.getData()
comb.saveCorrections(debug=True)
comb.mergeMovies(debug=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 36 Years of Box Office Mojo Movies
Found 97 Years of Wiki Film Movies
Found 93 Years of Rotten Tomatoes Movies
Found 67 Years of Ultimate Movie Rankings Movies
Found 38 Years of Razzies Movies
Found 84 Years of Oscar Movies
Found Data Between 1915 and 2017
Found 506 old corrections
Found 1 new corrections
Adding Larry The Cable Guy: Health Inspector
Saving data to /Users/tgadfort/Documents/code/movies/combine/data/saved.yaml
Saved data to /Users/tgadfort/Documents/code/movies/combine/data/saved.yaml
  --> This file is 25.1kB.
There are 507 total corrections
Found 2994 movies
	 ('Box Office Mojo', 674)
	 ('Rotten Tomatoes', 542)
	 ('Wiki Film', 527)
	 ('Oscar', 446)
	 ('Razzies', 413)
	 ('Ultimate Movie Rankings', 392)
There are 2879 final movies
Saving data to /Users/tgadfort/Documents/code/movies/combine/results/movies.json
Saved data to /Users/tgadfort/Documents/code/movies/combine/results/m

# Find Corrections

In [48]:
data = {}
from searchUtils import findNearest

savename = setFile(comb.getDataDir(), "saved.yaml")
savedData = getFile(savename)

movies = comb.getCombinedMovies()

repData = {}
for corrMovie,corrs in savedData.items():
    for corr in corrs:
        repData[corr] = corrMovie


        
movielist = [x[:-7] for x,name in movies.items()] # if name in ["Oscar", 'Rotten Tomatoes', 'Box Office']]
for i,movie in enumerate(movielist):
    if repData.get(movie):
        #print([movielist[i]],' -> ',[repData[movie]])
        movielist[i] = repData[movie]

cutoff = 0.9
#for key,name in movies.items():
#    movie = key[:-7]
for im,movie in enumerate(movielist):
    if movie.find("IMAX") != -1:
        continue
    if movie.find("re-issue") != -1:
        continue

    if im % 100 == 0:
        print(im,'/',len(movielist))
    if movie.endswith(' '):        
        if data.get(movie[:-1]) is None:
            print(len(data),'\t',movie[:-1])
            data[movie[:-1]] = set()
            data[movie[:-1]].add(movie)
        tmpList = list(filter(lambda a: a != movie[:-1], movielist))
        results = findNearest(movie[:-1], tmpList, 3, cutoff)
        if len(results) > 0:
            for value in results:
                data[movie[:-1]].add(value)
    else:
        tmpList = list(filter(lambda a: a != movie, movielist))    
        results = findNearest(movie, tmpList, 3, cutoff)
        if len(results) > 0:
            if data.get(movie) is None:
                print(len(data),'\t',movie)
                data[movie] = set()
            for value in results:
                data[movie].add(value)
                
    if len(data) > 300:
        break
        
for k in data.keys():
    data[k] = list(data[k])

savename = setFile(comb.getDataDir(), "corr.yaml")
saveFile(idata=data, ifile=savename, debug=True)

0 / 2877
100 / 2877
0 	 Dr. Jekyll And Mr. Hyde
1 	 Tarzan The Ape Man
200 / 2877
300 / 2877
400 / 2877
500 / 2877
2 	 Cheaper By The Dozen
3 	 Come Back, Little Sheba
4 	 Come Back Little Sheba
600 / 2877
5 	 Rebel Without A Cause
6 	 Rebel without A Cause
700 / 2877
800 / 2877
7 	 The Pink Panther
900 / 2877
1000 / 2877
8 	 Smokey And The Bandit
1100 / 2877
9 	 Alien
10 	 Rocky II
11 	 Smokey And The Bandit II
12 	 Tarzan, The Ape Man
1200 / 2877
13 	 Rocky III
14 	 Beverly Hills Cop
15 	 Back To The Future I
16 	 Rocky IV
17 	 Aliens
1300 / 2877
18 	 Crocodile Dundee
19 	 The Karate Kid Part II
20 	 Lethal Weapon 1
21 	 Beverly Hills Cop II
22 	 Crocodile Dundee II
23 	 The Karate Kid: Part III
24 	 The Karate Kid Part III
25 	 Lethal Weapon 2
26 	 Rocky V
1400 / 2877
27 	 Back To The Future II
28 	 Back To The Future III
29 	 Basic Instinct
30 	 Lethal Weapon 3
1500 / 2877
31 	 The Santa Clause
32 	 Toy Story
33 	 Dr. Jekyll And Ms. Hyde
34 	 Mission Impossible
1600 / 2877
35 	 Men

SyntaxError: invalid syntax (<ipython-input-54-96bc77a4e10e>, line 1)